# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""../../../../da…","""89107017380151…","""43069695278869…",1.2927e19,"""11138514466168…","""17205043942117…",false
"""../../../../da…","""16877333597133…","""14944412790986…",1.4856e19,"""11138514466168…","""11381818651902…",true
"""../../../../da…","""64916770439543…","""74581467874130…",5.6458e17,"""48724838344751…","""17205043942117…",false
"""../../../../da…","""94720753924796…","""14944412790986…",4.0723e18,"""45832127361851…","""17205043942117…",false
"""../../../../da…","""94443892155689…","""43069695278869…",1.4677e19,"""45832127361851…","""17101859440275…",true
"""../../../../da…","""34852003694136…","""74581467874130…",1.6330e19,"""12642642038986…",null,false
"""../../../../da…","""56983794965021…","""74581467874130…",8.7675e18,"""45832127361851…","""17101859440275…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_City': 'PREDICTOR_1',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""11683591212641…","""11649660574972…",1.2927e19,"""FirstMortgage3…","""31846277196766…","""Rejected"""
"""../../../../da…","""15276510953709…","""15117302346408…",1.4856e19,"""FirstMortgage3…","""15314783068981…","""Accepted"""
"""../../../../da…","""15312862879673…","""14208320946955…",5.6458e17,"""MoneyMarketSav…","""31846277196766…","""Rejected"""
"""../../../../da…","""96758014397149…","""15117302346408…",4.0723e18,"""BasicChecking""","""31846277196766…","""Rejected"""
"""../../../../da…","""10652791039338…","""11649660574972…",1.4677e19,"""BasicChecking""","""27687341692046…","""Accepted"""
"""../../../../da…","""12142904534595…","""14208320946955…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""87469997037595…","""14208320946955…",8.7675e18,"""BasicChecking""","""27687341692046…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""14101432411370…","""16004866363288…",1.2927e19,"""FirstMortgage3…","""96586825543814…","""Rejected"""
"""../../../../da…","""63868272428647…","""51048916873235…",1.4856e19,"""FirstMortgage3…","""54766247405129…","""Accepted"""
"""../../../../da…","""13239000540512…","""37506623227353…",5.6458e17,"""MoneyMarketSav…","""96586825543814…","""Rejected"""
"""../../../../da…","""50404000790867…","""51048916873235…",4.0723e18,"""BasicChecking""","""96586825543814…","""Rejected"""
"""../../../../da…","""15822616089475…","""16004866363288…",1.4677e19,"""BasicChecking""","""16954592453710…","""Accepted"""
"""../../../../da…","""88633331002132…","""37506623227353…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""17443735162882…","""37506623227353…",8.7675e18,"""BasicChecking""","""16954592453710…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_City=PREDICTOR_1
Customer_MaritalStatus=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""17750833180547…","""96025784446871…",1.2927e19,"""FirstMortgage3…","""65376196351362…","""Rejected"""
"""53097440064854…","""18443386959666…",1.4856e19,"""FirstMortgage3…","""91843718754584…","""Accepted"""
"""11546635845522…","""21637612306859…",5.6458e17,"""MoneyMarketSav…","""65376196351362…","""Rejected"""
"""98258881086474…","""18443386959666…",4.0723e18,"""BasicChecking""","""65376196351362…","""Rejected"""
"""10677819646787…","""96025784446871…",1.4677e19,"""BasicChecking""","""98198209548002…","""Accepted"""
"""16260288107973…","""21637612306859…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""45540809588860…","""21637612306859…",8.7675e18,"""BasicChecking""","""98198209548002…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'